In [1]:
import os
import os.path
import pandas as pd
import sys
sys.path.append('..')
from src.utils.data import writePandas, getPandas, getConfig
os.chdir('..')
os.getcwd()
import seaborn as sns
import matplotlib.pyplot as plt
data = getPandas('pat_data')
conf = getConfig('data')
used_inds = conf['indices']['pat']['train'] + conf['indices']['pat']['test']
used_inds = conf['indices']['pat']['train']
train_inds = conf['indices']['pat']['train']
test_inds = conf['indices']['pat']['test']
mednet_fe = getPandas('pat_resnet_8')
train_data = data.iloc[train_inds]
train_good_inds = train_data[train_data['CAT'] == 1].index
train_bad_inds = train_data[train_data['CAT'] == 0].index
train_good_mednet = mednet_fe.iloc[train_good_inds]
train_bad_mednet = mednet_fe.iloc[train_bad_inds]
test_data = data.iloc[test_inds]
test_good_inds = test_data[test_data['CAT'] == 1].index
test_bad_inds = test_data[test_data['CAT'] == 0].index
test_good_mednet = mednet_fe.iloc[test_good_inds]
test_bad_mednet = mednet_fe.iloc[test_bad_inds]
from scipy.stats import pearsonr, ttest_ind

In [6]:
cols = train_good_mednet.columns[1:]
for col in cols:
    train_good_fe = train_good_mednet[col].values
    train_bad_fe = train_bad_mednet[col].values
    test_good_fe = test_good_mednet[col].values
    test_bad_fe = test_bad_mednet[col].values
    p = ttest_ind(train_good_fe, train_bad_fe)[1]
    p_test = ttest_ind(test_good_fe, test_bad_fe)[1]
    if p < 0.05 and p_test < 0.05:
        print(col, p, p_test)
    #if col == 'resnet_13074' or col == 'resnet_12889' or col == 'resnet_810':
        #print(col, p, p_test)

resnet_783 0.03510273136028112 0.013755120780654643
resnet_1458 0.035473534549101036 0.00993941909479698
resnet_3456 0.03324383318136392 0.015874395974889197
resnet_5238 0.04251387923174229 0.02029126438997956
resnet_7020 0.0007507717033663511 0.0016160430632416812
resnet_11313 0.023103375543940784 0.020947505940246657
resnet_12555 0.031009674451086596 0.02104698837940386
